## Exampville Mode Choice Logsums

In this notebook, we will walk through the creation of logsums from a tour mode choice model.
This example will assume the reader is familiar with the mathematical basics of discrete choice
modeling generally, and will focus on the technical aspects of estimating the parameters
of a discrete choice model in Python using Larch.

In [1]:
import larch, numpy, pandas, os

To begin, we will re-load the data files from our tour mode choice example.

In [2]:
import larch.exampville
hh = pandas.read_csv( larch.exampville.files.hh )
pp = pandas.read_csv( larch.exampville.files.person )
tour = pandas.read_csv( larch.exampville.files.tour )
skims = larch.OMX( larch.exampville.files.skims, mode='r' )

We'll also load the saved model from the mode choice estimation.
(If you haven't run the mode choice estimation notebook already,
this file may not exist on your system, but you can create it 
by running that notebook.)

In [3]:
m = larch.read_metadata('/tmp/exampville_mode_choice.html')

We'll replicate the pre-processing used in the mode choice estimation,
to merge the household and person characteristics into the tours data,
add the index values for the home TAZ's, filter to include only 
work tours, and merge with the level of service skims.  (If this 
pre-processing was computationally expensive, it would probably have
been better to save the results to disk and reload them as needed,
but for this model these commands will run almost instantaneously.)

In [4]:
raw = tour.merge(hh, on='HHID').merge(pp, on=('HHID', 'PERSONID'))
raw["HOMETAZi"] = raw["HOMETAZ"] - 1
raw["DTAZi"] = raw["DTAZ"] - 1
raw = raw[raw.TOURPURP == 1]
raw = raw.join(
    skims.get_rc_dataframe(
        raw.HOMETAZi, raw.DTAZi,
    )
)

Then we bundle the raw data into the `larch.DataFrames` structure,
as we did for estimation, and attach this structure to the model
as its `dataservice`.

In [5]:
# For clarity, we can define numbers as names for modes
DA = 1
SR = 2
Walk = 3
Bike = 4
Transit = 5

In [6]:
dfs = larch.DataFrames(
    co=raw, 
    alt_codes=[DA,SR,Walk,Bike,Transit], 
    alt_names=['DA','SR','Walk','Bike','Transit'],
    ch_name='TOURMODE',
)

m.dataservice = dfs

We'll also initialize an DataFrame with zeros to hold the computed logsums.
This DataFrame will have one row for each case in our source data,
and a column for each possible destination zone.

In [7]:
logsums = pandas.DataFrame(
    data=0.0,
    index=raw.index, 
    columns=skims.TAZ_ID
)

The logsums from a `Model` can be computed using the `Model.logsums` method.
However, if we want the logsums for each possible destination, we'll need
to replace the part of our data that depends on the destination zone, 
writing in the appropriate values for each destination in turn.  
We can simply iterate over the
zones, which is a little bit slow but easy to code.  This isn't a big 
deal, as generating the logsums only needs to be done once after the 
mode choice model is finalized.

In [8]:
for destination_i, dtaz in enumerate(logsums.columns):
    
    # Get the LOS data for this destination
    new_data = skims.get_rc_dataframe(
        raw.HOMETAZi, destination_i,
    )
    
    # Write this data into the model's dataservice,
    # overwriting the existing values.
    dfs.data_co[new_data.columns] = new_data
    
    # Run the pre-processing to create the arrays 
    # needed for computation.
    m.load_data()
    
    # Lastly, compute the logsums and save them
    # to the new DataFrame.
    logsums[dtaz] = m.logsums()


We can review some of the output to make sure it appears reasonable.

In [9]:
logsums.head()

TAZ_ID,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
0,-2.935713,-2.660536,-1.745670,-2.229338,-2.403182,-1.624226,-2.689859,-2.704618,-3.258728,-2.718739,...,-1.702316,-1.717786,-2.340960,-2.760617,-1.029280,-2.249790,-2.048407,-2.160306,-2.286868,-1.959773
1,-2.935713,-2.660536,-1.745670,-2.229338,-2.403182,-1.624226,-2.689859,-2.704618,-3.258728,-2.718739,...,-1.702316,-1.717786,-2.340960,-2.760617,-1.029280,-2.249790,-2.048407,-2.160306,-2.286868,-1.959773
3,-2.935713,-2.660536,-1.745670,-2.229338,-2.403182,-1.624226,-2.689859,-2.704618,-3.258728,-2.718739,...,-1.702316,-1.717786,-2.340960,-2.760617,-1.029280,-2.249790,-2.048407,-2.160306,-2.286868,-1.959773
7,-3.046216,-2.770092,-1.823420,-2.320433,-2.494165,-1.705379,-2.785956,-2.806232,-3.365641,-2.831113,...,-1.784256,-1.797553,-2.430906,-2.863390,-1.100512,-2.343775,-2.133362,-2.247115,-2.380724,-2.047770
10,-3.051758,-2.775558,-1.827227,-2.324878,-2.498529,-1.709190,-2.790614,-2.811234,-3.370947,-2.836754,...,-1.788103,-1.801300,-2.435210,-2.868381,-1.103854,-2.348454,-2.137388,-2.251244,-2.385223,-2.052071


Then we can persist the logsums dataframe to disk, for use in the next
example, where we will estimate a destination choice model.

In [10]:
logsums.to_pickle('/tmp/logsums.pkl.gz')